# Immigration Data Analytics
### Data Engineering Capstone Project

#### Project Summary
Lot of people migrate to and from US every year, which creates a huge task of tracking each and every immigrant in the country difficult. Luckily a huge ammount of data gets generated for every immigrant, the data about their travel, destination, visa type etc. When combined this data with weather and demographic data would certainly give some insights about immigrants such as, the peak period when there are lot of application for immigration, which sate in US is prefered etc.

With data data available I have built a robust and scalable pipeline in Airflow using Amazon Web Services such as using S3, EMR and postgres.
this pipeline moves raw data to S3 and transforms it into a data model which lets the regulators trach individual immigrants easily.


In [1]:
import pandas as pd
import datetime
import os
import numpy as np
from math import isnan

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T


In [3]:

def getNullStatus(df):
    df_c = df.apply(lambda x: not x.isnull().values.any(), axis=0).T
    df_c = pd.DataFrame(df_c).rename(columns={0:"contains_null?"})
    df_s = pd.DataFrame(df.iloc[0,:]).rename(columns={0:"example_value"})
    df_f = df_c.join(df_s).reset_index(drop=False)
    
    return df_f


### Step 1: Scope the Project and Gather Data

#### Scope 
Lot of people migrate to and from US every year, which creates a huge task of tracking each and every immigrant in the country difficult. Luckily a huge ammount of data gets generated for every immigrant, the data about their travel, destination, visa type etc. When combined this data with weather and demographic data would certainly give some insights about immigrants such as, the peak period when there are lot of application for immigration, which sate in US is prefered etc.

With data data available I have built a robust and scalable pipeline in Airflow using Amazon Web Services such as using S3, EMR and postgres.
this pipeline moves raw data to S3 and transforms it into a data model which lets the regulators trach individual immigrants easily.

#### Describe and Gather Data 

##### SOURCE
- I94 Immigration Data: This data comes from the US National Tourism and Trade Office [Source](https://travel.trade.gov/research/reports/i94/historical/2016.html). This data records immigration records partitioned by month of every year.
- World temperature Data: This dataset comes from Kaggle [Source](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data). Includes temperature recordings of cities around the world for a period of time
- US City Demographic Data: This dataset comes from OpenSoft [Source](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/). Includes population formation of US states, like race and gender.
- Aiport Code table: [Source](https://datahub.io/core/airport-codes#data). Includes a collection of airport codes and their respective cities, countries around the world.

## Explore Data
Exploring each data for missing values, duplicate values etc.

### Immigration Data

In [15]:
# for data exploration we will use pandas

# immegration data (sample)
df_immData = pd.read_csv("immigration_data_sample.csv", sep = ",")
df_immData = df_immData.iloc[:,1:]
df_immData.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [16]:
print("##############################################\n")
print(f"Sample Data Rows: {df_immData.shape[0]},\nSample Data Columns: {df_immData.shape[1]}\n")
print("##############################################\n")
getNullStatus(df_immData)

##############################################

Sample Data Rows: 1000,
Sample Data Columns: 28

##############################################



,index,contains_null?,example_value
0,cicid,True,4.08432e+06
1,i94yr,True,2016
2,i94mon,True,4
3,i94cit,True,209
4,i94res,True,209
5,i94port,True,HHW
6,arrdate,True,20566
7,i94mode,True,1
8,i94addr,False,HI
9,depdate,False,20573


### Observations and fixes [*These fixes will be done in pyspark*]

- From obserrvation I found out that there are some abnormalities in Immigration data which needs to be fixed. Below are the observations about i94 data
    - the data contains unwanted characters in 
        - `[dtadfile, dtaddto]` column contains unwanted characters such as `(D/S)`
        - The documentation of i94 staes that there are 3 genders `[F, M, O]`, But in the data there is an extra character `x`. Replacing this `x` with `O`
    - sas date format for columns `[arrdate, depdate]`
        - convert these columns to datettime
    - change date format of `[dtadfile, dtaddto]`
    - country and city codes are given in description file `I94_SAS_Labels_Descriptions.SAS`
        - extract city codes and country codes
    


### US Cities Demographic data

In [17]:
df_demog = pd.read_csv("us-cities-demographics.csv", sep=";")
df_demog.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [18]:
print("##############################################\n")
print(f"Sample Data Rows: {df_demog.shape[0]},\nSample Data Columns: {df_demog.shape[1]}\n")
print("##############################################\n")
getNullStatus(df_demog)

##############################################

Sample Data Rows: 2891,
Sample Data Columns: 12

##############################################



,index,contains_null?,example_value
0,City,True,Silver Spring
1,State,True,Maryland
2,Median Age,True,33.8
3,Male Population,False,40601
4,Female Population,False,41862
5,Total Population,True,82463
6,Number of Veterans,False,1562
7,Foreign-born,False,30908
8,Average Household Size,False,2.6
9,State Code,True,MD


#### Observations and fixes [*These fixes will be done in pyspark*]
- This data is pretty straight-forward. Although, The `State` and `City` columns do contains null values. This can be fixed by combining the same information from airport data with this data so that we have all the cities in US.

### Airport Codes

In [19]:
df_airport = pd.read_csv("airport-codes_csv.csv", sep=',')
df_airport.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [28]:
print("##############################################\n")
print(f"Sample Data Rows: {df_airport.shape[0]},\nSample Data Columns: {df_airport.shape[1]}\n")
print("##############################################\n")
getNullStatus(df_airport)

##############################################

Sample Data Rows: 55075,
Sample Data Columns: 12

##############################################



,index,contains_null?,example_value
0,ident,True,00A
1,type,True,heliport
2,name,True,Total Rf Heliport
3,elevation_ft,False,11
4,continent,False,NaN
5,iso_country,False,US
6,iso_region,True,US-PA
7,municipality,False,Bensalem
8,gps_code,False,00A
9,iata_code,False,NaN


#### Observations and fixed [*These fixes will be done in pyspark*]

- The column `coordinates` needs a fix, the values of lattitude and longitude are combined in one column. These values can be seperated.
- state and country codes are combined into one in column `iso_region`

#### Global temperature by cities

In [36]:
df_gtemp = pd.read_csv("/data2/GlobalLandTemperaturesByCity.csv", sep=',', nrows=10000)
df_gtemp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [37]:
print("##############################################\n")
print(f"Sample Data Rows: {df_gtemp.shape[0]},\nSample Data Columns: {df_gtemp.shape[1]}\n")
print("##############################################\n")
getNullStatus(df_gtemp)

##############################################

Sample Data Rows: 10000,
Sample Data Columns: 7

##############################################



,index,contains_null?,example_value
0,dt,True,1743-11-01
1,AverageTemperature,False,6.068
2,AverageTemperatureUncertainty,False,1.737
3,City,True,Århus
4,Country,True,Denmark
5,Latitude,True,57.05N
6,Longitude,True,10.33E


The data pipeline is implemented in airflow, for more reference read `Readme.md` file.